In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from ActivationFunction import *
from Net.Layer import *
from Net.FullConnectedNeuralNetworkBProp import *
from Net.FullConnectedNeuralNetworkRProp import *
from ActivationFunction.ActivationFunction import * 

In [8]:
test_data_pd = pd.read_csv("./Dataset/mnist_test.csv")
train_data_pd = pd.read_csv("./Dataset/mnist_train_small.csv")
columns = ["label"]
pixels = [f"pixel {pixel}" for pixel in range(1, 785)]
columns = columns + pixels

test_data_pd.columns = columns
train_data_pd.columns = columns

print(f"# Campioni di training: {len(train_data_pd)}; # Campioni di test: {len(test_data_pd)}")

def one_hot_encoding(Y):
  one_hot_Y = np.zeros((Y.size, Y.max() + 1))
  one_hot_Y[np.arange(Y.size), Y] = 1
  return one_hot_Y.T

# Campioni di training: 19999; # Campioni di test: 9999


In [9]:
train_data = np.array(train_data_pd)
test_data = np.array(test_data_pd)

# piccolo shuffle di cortesia
np.random.shuffle(train_data)
np.random.shuffle(test_data)
rows, cols = test_data.shape

val_data = test_data[0:2500]
# test_data = test_data[2500:rows+1]

# X_train = train_data[:, 1:train_data.shape[1]+1].T
X_train = train_data[:100, 1:train_data.shape[1]+1].T
# Y_train = train_data[:, 0]
Y_train = train_data[:100, 0]
Y_train = one_hot_encoding(Y_train)

X_test = test_data[:, 1:test_data.shape[1]+1].T
Y_test = test_data[:, 0]
Y_test = one_hot_encoding(Y_test)

X_val = val_data[:, 1:val_data.shape[1]+1]
Y_val = val_data[:, 0]

print(X_train.shape)
print(Y_train.shape)

(784, 100)
(10, 100)


In [4]:
input_layer = Layer(784, 256, SigmoidActivationFunction)
output_layer = Layer(256, 10, SoftMaxActivationFunction)
layers = [input_layer, output_layer]

nn = FullConnectedNeuralNetworkBprop(layers, SumOfSquaredError, learning_rate = 0.01)

In [ ]:
nn.train(X_train, Y_train, validation_set=X_test, validation_targets=Y_test, epochs=20, training_method=TrainingMethod.BATCH)
print(f"Final accuracy = {nn.accuracy}")

In [6]:
input_layer = Layer(784, 256, SigmoidActivationFunction)
output_layer = Layer(256, 10, SoftMaxActivationFunction)
layers = [input_layer, output_layer]

rprop_nn = FullConnectedNeuralNetworkRprop(layers, SumOfSquaredError)
rprop_nn.train(X_train, Y_train, validation_set=X_test, validation_targets=Y_test, epochs=50)

Working with 100 samples
Fine epoch #0; mean_train_error = 0.47932634 - mean_val_error: 0.47875222 - accuracy = 0.12000000
Fine epoch #1; mean_train_error = 0.86000000 - mean_val_error: 0.89728973 - accuracy = 0.14120000
Fine epoch #2; mean_train_error = 0.89999766 - mean_val_error: 0.90418796 - accuracy = 0.10141200


KeyboardInterrupt: 

In [ ]:
not_encoded_rprop_nn = FullConnectedNeuralNetworkRprop([Layer(784, 256, SigmoidActivationFunction),
                                                        Layer(256, 10, SoftMaxActivationFunction)],
                                                        CrossEntropy,
                                                        learning_rate = 1,
                                                        rprop_eta_minus=0.0001,
                                                        rprop_eta_plus=2
                                                        )
not_encoded_rprop_nn.train(X_train, Y_train, validation_set=X_test, validation_targets=Y_test, epochs=150)

In [ ]:
encoder_size = [627, 470, 392, 313, 156]
number_of_epoch_encoder = 300
number_of_training_epochs = 150
encoder_layer_size = encoder_size[0]
metrics = {}

In [ ]:
rprop_encoder = FullConnectedNeuralNetworkBprop([Layer(784, encoder_layer_size, LeakyRELUActivationFunction),
                                                 Layer(encoder_layer_size, 784, RELUActivationFunction)
                                                 ], SumOfSquaredError, 0.00005)

rprop_encoder.train(X_train, X_train, number_of_epoch_encoder)

In [ ]:
sample_index = 2
plt.imshow(X_train[:, sample_index].reshape([28, 28]))
plt.show()
plt.imshow(rprop_encoder.predict(X_train[:, sample_index]).reshape([28, 28]))
plt.show()

In [ ]:
encoder = FullConnectedNeuralNetwork([rprop_encoder.layers[0]], SumOfSquaredError)

X_encoded_train = np.empty([encoder_layer_size, X_train.shape[1]])
Y_encoded_train = Y_train
for X_sample in X_train.T:
  X_encoded_train[:, 0] = encoder.predict(X_sample)

print(X_encoded_train.shape)
print(X_encoded_train.shape)

X_encoded_test = np.empty([encoder_layer_size, X_test.shape[1]])
Y_encoded_test = Y_test
for X_sample in X_test.T:
  X_encoded_test[:, 0] = encoder.predict(X_sample)

print(X_encoded_test.shape)
print(X_encoded_test.shape)

In [ ]:
encoded_rprop_nn = FullConnectedNeuralNetworkRprop([Layer(encoder_layer_size, 256, SigmoidActivationFunction),
                                                    Layer(256, 10, SoftMaxActivationFunction)],
                                                   CrossEntropy,
                                                   )
encoded_rprop_nn.train(X_encoded_train, Y_encoded_train, validation_set=X_encoded_test, validation_targets=Y_encoded_test, epochs=20)


In [ ]:
encoded_rprop_nn = FullConnectedNeuralNetworkRprop([Layer(encoder_layer_size, 256, SigmoidActivationFunction),
                                                    Layer(256, 10, SoftMaxActivationFunction)],
                                                   CrossEntropy,
                                                   learning_rate = 1,
                                                   rprop_eta_minus=0.0001,
                                                   rprop_eta_plus=2
                                                    )
encoded_rprop_nn.train(X_encoded_train, Y_encoded_train, validation_set=X_encoded_test, validation_targets=Y_encoded_test, number_of_training_epochs=20)

metrics[f"{encoder_layer_size}"] = {"train_errors":encoded_rprop_nn.mean_train_error,
                                    "val_errors":encoded_rprop_nn.mean_val_error,
                                    "accuracy":encoded_rprop_nn.accuracies}

In [ ]:
encoder_layer_size = encoder_size[0]
rprop_encoder = FullConnectedNeuralNetworkBprop([Layer(784, encoder_layer_size, LeakyRELUActivationFunction),
                                                 Layer(encoder_layer_size, 784, RELUActivationFunction)
                                                 ], SumOfSquaredError, 0.00005)

rprop_encoder.train(X_train[], X_train[], number_of_epoch_encoder)

sample_index = 1
plt.imshow(X_train[:, sample_index].reshape([28, 28]))
plt.show()
plt.imshow(rprop_encoder.predict(X_train[:, sample_index]).reshape([28, 28]))
plt.show()

encoder = FullConnectedNeuralNetwork([rprop_encoder.layers[0]], SumOfSquaredError)

X_encoded_train = np.empty([encoder_layer_size, X_train.shape[1]])
Y_encoded_train = Y_train
for X_sample in X_train.T:
  X_encoded_train[:, 0] = encoder.predict(X_sample)

print(X_encoded_train.shape)
print(X_encoded_train.shape)

X_encoded_test = np.empty([encoder_layer_size, X_test.shape[1]])
Y_encoded_test = Y_test
for X_sample in X_test.T:
  X_encoded_test[:, 0] = encoder.predict(X_sample)

print(X_encoded_test.shape)
print(X_encoded_test.shape)

encoded_rprop_nn = FullConnectedNeuralNetworkRprop([Layer(encoder_layer_size, 256, SigmoidActivationFunction),
                                                    Layer(256, 10, SoftMaxActivationFunction)],
                                                   CrossEntropy,
                                                   learning_rate = 1,
                                                   rprop_eta_minus=0.0001,
                                                   rprop_eta_plus=2
                                                    )
encoded_rprop_nn.train(X_encoded_train, Y_encoded_train, validation_set=X_encoded_test, validation_targets=Y_encoded_test, number_of_training_epochs=20)

metrics[f"{encoder_layer_size}"] = {"train_errors":encoded_rprop_nn.mean_train_error,
                                    "val_errors":encoded_rprop_nn.mean_val_error,
                                    "accuracy":encoded_rprop_nn.accuracies}

In [ ]:
encoder_layer_size = encoder_size[1]
rprop_encoder = FullConnectedNeuralNetworkBprop([Layer(784, encoder_layer_size, LeakyRELUActivationFunction),
                                                 Layer(encoder_layer_size, 784, RELUActivationFunction)
                                                 ], SumOfSquaredError, 0.00005)

rprop_encoder.train(X_train, X_train, number_of_epoch_encoder)

sample_index = 1
plt.imshow(X_train[:, sample_index].reshape([28, 28]))
plt.show()
plt.imshow(rprop_encoder.predict(X_train[:, sample_index]).reshape([28, 28]))
plt.show()

encoder = FullConnectedNeuralNetwork([rprop_encoder.layers[0]], SumOfSquaredError)

X_encoded_train = np.empty([encoder_layer_size, X_train.shape[1]])
Y_encoded_train = Y_train
for X_sample in X_train.T:
  X_encoded_train[:, 0] = encoder.predict(X_sample)

print(X_encoded_train.shape)
print(X_encoded_train.shape)

X_encoded_test = np.empty([encoder_layer_size, X_test.shape[1]])
Y_encoded_test = Y_test
for X_sample in X_test.T:
  X_encoded_test[:, 0] = encoder.predict(X_sample)

print(X_encoded_test.shape)
print(X_encoded_test.shape)

encoded_rprop_nn = FullConnectedNeuralNetworkRprop([Layer(encoder_layer_size, 256, SigmoidActivationFunction),
                                                    Layer(256, 10, SoftMaxActivationFunction)],
                                                   CrossEntropy,
                                                   learning_rate = 1,
                                                   rprop_eta_minus=0.0001,
                                                   rprop_eta_plus=2
                                                    )
encoded_rprop_nn.train(X_encoded_train, Y_encoded_train, validation_set=X_encoded_test, validation_targets=Y_encoded_test, number_of_training_epochs=20)

metrics[f"{encoder_layer_size}"] = {"train_errors":encoded_rprop_nn.mean_train_error,
                                    "val_errors":encoded_rprop_nn.mean_val_error,
                                    "accuracy":encoded_rprop_nn.accuracies}

In [ ]:
encoder_layer_size = encoder_size[2]
rprop_encoder = FullConnectedNeuralNetworkBprop([Layer(784, encoder_layer_size, LeakyRELUActivationFunction),
                                                 Layer(encoder_layer_size, 784, RELUActivationFunction)
                                                 ], SumOfSquaredError, 0.00005)

rprop_encoder.train(X_train, X_train, number_of_epoch_encoder)

sample_index = 1
plt.imshow(X_train[:, sample_index].reshape([28, 28]))
plt.show()
plt.imshow(rprop_encoder.predict(X_train[:, sample_index]).reshape([28, 28]))
plt.show()

encoder = FullConnectedNeuralNetwork([rprop_encoder.layers[0]], SumOfSquaredError)

X_encoded_train = np.empty([encoder_layer_size, X_train.shape[1]])
Y_encoded_train = Y_train
for X_sample in X_train.T:
  X_encoded_train[:, 0] = encoder.predict(X_sample)

print(X_encoded_train.shape)
print(X_encoded_train.shape)

X_encoded_test = np.empty([encoder_layer_size, X_test.shape[1]])
Y_encoded_test = Y_test
for X_sample in X_test.T:
  X_encoded_test[:, 0] = encoder.predict(X_sample)

print(X_encoded_test.shape)
print(X_encoded_test.shape)

encoded_rprop_nn = FullConnectedNeuralNetworkRprop([Layer(encoder_layer_size, 256, SigmoidActivationFunction),
                                                    Layer(256, 10, SoftMaxActivationFunction)],
                                                   CrossEntropy,
                                                   learning_rate = 1,
                                                   rprop_eta_minus=0.0001,
                                                   rprop_eta_plus=2
                                                    )
encoded_rprop_nn.train(X_encoded_train, Y_encoded_train, validation_set=X_encoded_test, validation_targets=Y_encoded_test, number_of_training_epochs=20)

metrics[f"{encoder_layer_size}"] = {"train_errors":encoded_rprop_nn.mean_train_error,
                                    "val_errors":encoded_rprop_nn.mean_val_error,
                                    "accuracy":encoded_rprop_nn.accuracies}

In [ ]:
encoder_layer_size = encoder_size[3]
rprop_encoder = FullConnectedNeuralNetworkBprop([Layer(784, encoder_layer_size, LeakyRELUActivationFunction),
                                                 Layer(encoder_layer_size, 784, RELUActivationFunction)
                                                 ], SumOfSquaredError, 0.00005)

rprop_encoder.train(X_train, X_train, number_of_epoch_encoder)

sample_index = 1
plt.imshow(X_train[:, sample_index].reshape([28, 28]))
plt.show()
plt.imshow(rprop_encoder.predict(X_train[:, sample_index]).reshape([28, 28]))
plt.show()

encoder = FullConnectedNeuralNetwork([rprop_encoder.layers[0]], SumOfSquaredError)

X_encoded_train = np.empty([encoder_layer_size, X_train.shape[1]])
Y_encoded_train = Y_train
for X_sample in X_train.T:
  X_encoded_train[:, 0] = encoder.predict(X_sample)

print(X_encoded_train.shape)
print(X_encoded_train.shape)

X_encoded_test = np.empty([encoder_layer_size, X_test.shape[1]])
Y_encoded_test = Y_test
for X_sample in X_test.T:
  X_encoded_test[:, 0] = encoder.predict(X_sample)

print(X_encoded_test.shape)
print(X_encoded_test.shape)

encoded_rprop_nn = FullConnectedNeuralNetworkRprop([Layer(encoder_layer_size, 256, SigmoidActivationFunction),
                                                    Layer(256, 10, SoftMaxActivationFunction)],
                                                   CrossEntropy,
                                                   learning_rate = 1,
                                                   rprop_eta_minus=0.0001,
                                                   rprop_eta_plus=2
                                                    )
encoded_rprop_nn.train(X_encoded_train, Y_encoded_train, validation_set=X_encoded_test, validation_targets=Y_encoded_test, number_of_training_epochs=20)

metrics[f"{encoder_layer_size}"] = {"train_errors":encoded_rprop_nn.mean_train_error,
                                    "val_errors":encoded_rprop_nn.mean_val_error,
                                    "accuracy":encoded_rprop_nn.accuracies}

In [ ]:
encoder_layer_size = encoder_size[4]
rprop_encoder = FullConnectedNeuralNetworkBprop([Layer(784, encoder_layer_size, LeakyRELUActivationFunction),
                                                 Layer(encoder_layer_size, 784, RELUActivationFunction)
                                                 ], SumOfSquaredError, 0.00005)

rprop_encoder.train(X_train, X_train, number_of_epoch_encoder)

sample_index = 1
plt.imshow(X_train[:, sample_index].reshape([28, 28]))
plt.show()
plt.imshow(rprop_encoder.predict(X_train[:, sample_index]).reshape([28, 28]))
plt.show()

encoder = FullConnectedNeuralNetwork([rprop_encoder.layers[0]], SumOfSquaredError)

X_encoded_train = np.empty([encoder_layer_size, X_train.shape[1]])
Y_encoded_train = Y_train
for X_sample in X_train.T:
  X_encoded_train[:, 0] = encoder.predict(X_sample)

print(X_encoded_train.shape)
print(X_encoded_train.shape)

X_encoded_test = np.empty([encoder_layer_size, X_test.shape[1]])
Y_encoded_test = Y_test
for X_sample in X_test.T:
  X_encoded_test[:, 0] = encoder.predict(X_sample)

print(X_encoded_test.shape)
print(X_encoded_test.shape)

encoded_rprop_nn = FullConnectedNeuralNetworkRprop([Layer(encoder_layer_size, 256, SigmoidActivationFunction),
                                                    Layer(256, 10, SoftMaxActivationFunction)],
                                                   CrossEntropy,
                                                   learning_rate = 1,
                                                   rprop_eta_minus=0.0001,
                                                   rprop_eta_plus=2
                                                    )
encoded_rprop_nn.train(X_encoded_train, Y_encoded_train, validation_set=X_encoded_test, validation_targets=Y_encoded_test, number_of_training_epochs=20)

metrics[f"{encoder_layer_size}"] = {"train_errors":encoded_rprop_nn.mean_train_error,
                                    "val_errors":encoded_rprop_nn.mean_val_error,
                                    "accuracy":encoded_rprop_nn.accuracies}